In [ ]:
import json
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from typing import Dict, List, Any, Tuple

In [ ]:
class RuleBasedScoringEngine:
    """
    A deterministic rule-based scoring engine that applies traditional palmistry rules
    to generate quantitative scores based on textual descriptions.
    """
    
    def __init__(self, rule_config: Dict[str, Any] = None):
        self.rules = rule_config or self._get_default_rules()
        

    def _get_default_rules(self) -> Dict[str, List]:
        """Define default palmistry rules based on traditional knowledge."""
        return {
            "life_line_rules": [
                {"condition": "long and curved", "scores": {"strength": 0.8, "romantic": 0.6, "luck": 0.7, "potential": 0.9}},
                {"condition": "short and straight", "scores": {"strength": 0.4, "romantic": 0.3, "luck": 0.5, "potential": 0.6}},
                {"condition": "deep and clear", "scores": {"strength": 0.9, "romantic": 0.7, "luck": 0.8, "potential": 0.8}},
                {"condition": "faint and broken", "scores": {"strength": 0.3, "romantic": 0.4, "luck": 0.2, "potential": 0.4}}
            ],
            "heart_line_rules": [
                {"condition": "long and curved", "scores": {"strength": 0.6, "romantic": 0.9, "luck": 0.7, "potential": 0.7}},
                {"condition": "straight and short", "scores": {"strength": 0.7, "romantic": 0.4, "luck": 0.6, "potential": 0.5}},
                {"condition": "deep and clear", "scores": {"strength": 0.8, "romantic": 0.8, "luck": 0.7, "potential": 0.8}},
                {"condition": "chain-like", "scores": {"strength": 0.5, "romantic": 0.9, "luck": 0.6, "potential": 0.6}}
            ],
            "head_line_rules": [
                {"condition": "long and straight", "scores": {"strength": 0.7, "romantic": 0.5, "luck": 0.6, "potential": 0.9}},
                {"condition": "short and curved", "scores": {"strength": 0.5, "romantic": 0.7, "luck": 0.5, "potential": 0.6}},
                {"condition": "deep and clear", "scores": {"strength": 0.8, "romantic": 0.6, "luck": 0.7, "potential": 0.8}},
                {"condition": "faint and wavy", "scores": {"strength": 0.4, "romantic": 0.5, "luck": 0.4, "potential": 0.5}}
            ],
            "fate_line_rules": [
                {"condition": "present and clear", "scores": {"strength": 0.7, "romantic": 0.6, "luck": 0.8, "potential": 0.8}},
                {"condition": "absent", "scores": {"strength": 0.5, "romantic": 0.5, "luck": 0.5, "potential": 0.5}},
                {"condition": "deep and long", "scores": {"strength": 0.8, "romantic": 0.7, "luck": 0.9, "potential": 0.9}},
                {"condition": "broken", "scores": {"strength": 0.4, "romantic": 0.4, "luck": 0.3, "potential": 0.4}}
            ]
        }
    

    def parse_text_descriptions(self, text_description: str) -> Dict[str, str]:
        """
        Parse the text description from the Text Decoder branch to extract line characteristics.
        """
        characteristics = {}
        lines = text_description.split(';')
        for line in lines:
            if ':' in line:
                key, value = line.split(':', 1)
                characteristics[key.strip()] = value.strip()
        return characteristics
    

    def apply_rules(self, characteristics: Dict[str, str]) -> Dict[str, float]:
        """
        Apply rules based on the extracted characteristics to generate scores.
        """
        scores = {"strength": 0.0, "romantic": 0.0, "luck": 0.0, "potential": 0.0}
        weights = {"life_line": 0.3, "heart_line": 0.3, "head_line": 0.25, "fate_line": 0.15}
        rule_count = {key: 0 for key in scores.keys()}
        
        for line_type, description in characteristics.items():
            if line_type in self.rules and description:
                line_rules = self.rules[line_type]
                weight = weights.get(line_type, 0.2)
                
                for rule in line_rules:
                    if rule["condition"] in description.lower():
                        for score_type, score_value in rule["scores"].items():
                            scores[score_type] += score_value * weight
                            rule_count[score_type] += 1
        
        # normalize scores by rule count
        for score_type in scores:
            if rule_count[score_type] > 0:
                scores[score_type] = min(1.0, scores[score_type] / rule_count[score_type])
            else:
                scores[score_type] = 0.5  # default neutral score
        
        return scores
    
    
    def calculate_scores(self, text_description: str) -> Dict[str, float]:
        """Main method to calculate rule-based scores from text descriptions."""
        characteristics = self.parse_text_descriptions(text_description)
        return self.apply_rules(characteristics)


In [ ]:
class PalmistryScorePredictor(nn.Module):
    """
    Neural network for predicting palmistry scores from unified embeddings.
    """
    
    def __init__(self, embedding_dim: int = 512, hidden_dims: List[int] = None):
        super(PalmistryScorePredictor, self).__init__()
        self.hidden_dims = hidden_dims or [256, 128, 64]
        
        layers = []
        input_dim = embedding_dim
        
        for hidden_dim in self.hidden_dims:
            layers.extend([
                nn.Linear(input_dim, hidden_dim),
                nn.ReLU(),
                nn.Dropout(0.3),
                nn.BatchNorm1d(hidden_dim)
            ])
            input_dim = hidden_dim
        
        layers.append(nn.Linear(input_dim, 4))  # 4 scores: "strength", "romantic", "luck", "potential"
        layers.append(nn.Sigmoid())             # sigmoid to control range to be [0, 1]
        
        self.network = nn.Sequential(*layers)
    

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.network(x)

In [ ]:
class NeuralNetworkScorePredictor:
    """
    A shallow neural network that predicts scores from unified embeddings.
    Correlates with rule-based scores while learning non-linear patterns.
    """
    
    def __init__(self, embedding_dim: int = 512, hidden_dims: List[int] = None):
        self.embedding_dim = embedding_dim
        self.model = PalmistryScorePredictor(embedding_dim, hidden_dims)
        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.001)
        

    def train(self, embeddings: np.ndarray, rule_scores: np.ndarray, 
              validation_data: Tuple = None, epochs: int = 100, 
              batch_size: int = 32) -> Dict[str, List[float]]:
        """
        Train the neural network to predict scores that correlate with rule-based scores.
        """

        X_train = torch.FloatTensor(embeddings)
        y_train = torch.FloatTensor(rule_scores)
        
        dataset = TensorDataset(X_train, y_train)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        
        self.model.train()
        
        train_losses = []
        val_losses = []
        
        for epoch in range(epochs):
            epoch_loss = 0.0
            for batch_X, batch_y in dataloader:
                self.optimizer.zero_grad()
                
                # forward pass
                outputs = self.model(batch_X)
                loss = self.criterion(outputs, batch_y)
                
                # backward pass
                loss.backward()
                self.optimizer.step()
                
                epoch_loss += loss.item()
            
            avg_train_loss = epoch_loss / len(dataloader)
            train_losses.append(avg_train_loss)
            
            # Validation if validation data provided
            if validation_data is not None:
                val_loss = self.evaluate(validation_data[0], validation_data[1])
                val_losses.append(val_loss)
                
                if epoch % 10 == 0:
                    print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {val_loss:.4f}')
            else:
                if epoch % 10 == 0:
                    print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {avg_train_loss:.4f}')
        
        return {'train_loss': train_losses, 'val_loss': val_losses}
    

    def evaluate(self, X_val: np.ndarray, y_val: np.ndarray) -> float:
        """Evaluate the model on validation data."""
        self.model.eval()
        with torch.no_grad():
            X_val_tensor = torch.FloatTensor(X_val)
            y_val_tensor = torch.FloatTensor(y_val)
            outputs = self.model(X_val_tensor)
            loss = self.criterion(outputs, y_val_tensor)
        self.model.train()
        return loss.item()
    
    
    def predict_scores(self, embedding: np.ndarray) -> Dict[str, float]:
        """Predict scores from a single embedding vector."""
        self.model.eval()
        
        # Ensure correct input format
        if isinstance(embedding, np.ndarray):
            embedding = torch.FloatTensor(embedding)
        
        if len(embedding.shape) == 1:
            embedding = embedding.unsqueeze(0)
        
        with torch.no_grad():
            predictions = self.model(embedding)
            scores_array = predictions[0].numpy()  # first prediction
        
        return {
            "strength": float(scores_array[0]),
            "romantic": float(scores_array[1]),
            "luck": float(scores_array[2]),
            "potential": float(scores_array[3])
        }
    

    def save_model(self, filepath: str):
        """Save the trained model."""
        torch.save({
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'embedding_dim': self.embedding_dim
        }, filepath)
    

    def load_model(self, filepath: str):
        """Load a pre-trained model."""
        checkpoint = torch.load(filepath)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [ ]:
class HybridScoringSystem:
    """
    Combined system that integrates rule-based and neural network scoring approaches.
    Provides final scores with configurable weighting between the two approaches.
    """
    
    def __init__(self, embedding_dim: int = 512, alpha: float = 0.7):
        """
        Initialize the hybrid scoring system.
        
        args:
        - embedding_dim: Dimension of the input embedding vector
        - alpha: Weight for rule-based scores (1-alpha for neural network scores)
        """
        self.rule_engine = RuleBasedScoringEngine()
        self.nn_predictor = NeuralNetworkScorePredictor(embedding_dim=embedding_dim)
        self.alpha = alpha
    

    def set_alpha(self, alpha: float):
        """Set the weighting parameter between rule-based and NN scores."""
        self.alpha = max(0.0, min(1.0, alpha))
    

    def calculate_final_scores(self, text_description: str, 
                              embedding: np.ndarray) -> Dict[str, float]:
        """
        Calculate final scores by combining rule-based and neural network predictions.
        """
        rule_scores = self.rule_engine.calculate_scores(text_description)   # rule-based scores
        
        nn_scores = self.nn_predictor.predict_scores(embedding)             # predicted scores from NN
        
        final_scores = {}
        for key in rule_scores.keys():
            rule_score = rule_scores[key]
            nn_score = nn_scores[key]
            final_scores[key] = (self.alpha * rule_score + 
                               (1 - self.alpha) * nn_score)                 # combines scores by weighted average
        
        return final_scores
    

    def get_detailed_breakdown(self, text_description: str, 
                             embedding: np.ndarray) -> Dict[str, Any]:
        """Get detailed scoring breakdown including individual components."""
        rule_scores = self.rule_engine.calculate_scores(text_description)
        nn_scores = self.nn_predictor.predict_scores(embedding)
        final_scores = self.calculate_final_scores(text_description, embedding)
        
        return {
            "rule_based_scores": rule_scores,
            "neural_network_scores": nn_scores,
            "final_scores": final_scores,
            "weight_parameter": self.alpha
        }

In [ ]:
def train_model():
    """Example training function."""
    
    embedding_dim = 512
    predictor = NeuralNetworkScorePredictor(embedding_dim=embedding_dim)
    
    num_samples = 1000
    X_train = np.random.randn(num_samples, embedding_dim).astype(np.float32)    # example: random data
    
    rule_scores = np.random.rand(num_samples, 4).astype(np.float32)             # example: random number as scores
    
    history = predictor.train(X_train, rule_scores, epochs=50, batch_size=32)
    
    print("Training completed!")
    return predictor, history

In [ ]:
def demonstrate_system():
    """Demonstrate the scoring system with example data."""
    
    embedding_dim = 512
    scoring_system = HybridScoringSystem(embedding_dim=embedding_dim)
    
    example_text = "life_line: long and curved; heart_line: deep and clear; head_line: long and straight; fate_line: present and clear"
    
    example_embedding = np.random.randn(embedding_dim).astype(np.float32)
    
    final_scores = scoring_system.calculate_final_scores(example_text, example_embedding)
    
    print("=== PALMISTRY SCORING SYSTEM RESULTS ===")
    print("\nFinal Scores:")
    print(json.dumps(final_scores, indent=2, ensure_ascii=False))
    
    return final_scores

In [ ]:
final_scores = demonstrate_system()

trained_predictor, training_history = train_model()